In [1]:
import asyncio
import websockets
import json
import time

import sys
sys.path.append("./chainklik")
import pandas as pd
import requests
from web3 import Web3
from collections import defaultdict 

import libs.common.utils as utils
import libs.common.payload as payload
import config.config as cfg
from nodes.data import *
from nodes.eth import *
from nodes.condition import *
from nodes.computation import *

In [2]:
# Connect to an Ethereum node
w3 = Web3(Web3.HTTPProvider(cfg.config["eth_sepolia_http_url"]))

# Set sender and recipient addresses
sender_address = '0x5bA4D4264Bf9A8C3aaF7e1fea6f83f50643A3Fd7'
recipient_address = '0xaf6667a2F847beeca6a6604126Dc28344518840b'

# Set private key for the sender's account. 
private_key = '1dab201501e8b882ca3413edcdfed263e4834cd8ea4c9586aca7fb699c51d681'

In [3]:
balance_sender = w3.from_wei(w3.eth.get_balance(sender_address), 'ether')
balance_recipient = w3.from_wei(w3.eth.get_balance(recipient_address), 'ether')

print(f'The balance of { sender_address } is: { balance_sender } ETH')
print(f'The balance of { recipient_address } is: { balance_recipient } ETH')

The balance of 0x5bA4D4264Bf9A8C3aaF7e1fea6f83f50643A3Fd7 is: 0.169946913897967 ETH
The balance of 0xaf6667a2F847beeca6a6604126Dc28344518840b is: 0.2485 ETH


In [4]:
w3.eth.get_block("latest")["number"]

5034195

In [5]:
bt = BlockTimer(w3.eth.get_block("latest")["number"], w3.eth.get_block("latest")["number"]+50, 10, "eth::sepolia")
data_node = DataNode("data", {}, {}, bt)

In [6]:
tx_node1 = TxNode("tx1",{"in":"data"},{"wallet":sender_address,"to":recipient_address,"amount":0.001,"chain":"eth::sepolia"}, w3, private_key)
tx_node2 = TxNode("tx2",{"in":"data"},{"wallet":sender_address,"to":recipient_address,"amount":0.002,"chain":"eth::sepolia"}, w3, private_key)

In [7]:
cum_node = CumNode("cum",{"in":"tx1"},{})

In [8]:
comp_node = ThresholdNode("comp",{"in":"cum"},{"th":0.002,"condition":"ge"})

In [9]:
tx_node3 = TxNode("tx3",{"in":"comp"},{"wallet":sender_address,"to":recipient_address,"amount":0.003,"chain":"eth::sepolia"}, w3, private_key)

In [10]:
node_layers = [
    ["data"],
    ["tx1", "tx2"],
    ["cum"],
    ["comp"],
    ["tx3"]
]

In [11]:
nodes = {
    "data": data_node, 
    "tx1": tx_node1,
    "tx2": tx_node2,
    "cum": cum_node,
    "comp": comp_node,
    "tx3": tx_node3
}

In [12]:
nodes["comp"].__dict__

{'id': 'comp',
 'deps': {'in': 'cum'},
 'params': {'th': 0.002, 'condition': 'ge'},
 'output': None,
 'active': False,
 'finalized': True}

In [13]:
def print_nodes_state(node_layers):
    print('######## node state ##########')
    for node_layer in node_layers:
        print([(nodes[key].active,nodes[key].finalized,nodes[key].output) for key in node_layer])

In [14]:
ctx = {"eth::sepolia":{}}
cache = {}

In [15]:
async def process_message(block_info):
    ctx["eth::sepolia"]["block_time"] = int(block_info["number"], 16)
    print(ctx)
    
    for key in nodes:
        if nodes[key].finalized:
            nodes[key].active = False
        else:
            nodes[key].active = True

    print_nodes_state(node_layers)
    
    for node_layer in node_layers:
        for key in node_layer:
            values = {}
            if len(nodes[key].deps) == 0: # no deps/input node
                nodes[key].run(ctx, values)
            else:
                is_deps_finalized = True
                is_deps_active = False
                for dep in nodes[key].deps:
                    is_deps_finalized = is_deps_finalized and nodes[nodes[key].deps[dep]].finalized
                    is_deps_active = is_deps_active or nodes[nodes[key].deps[dep]].active
                    values[dep] = cache[nodes[key].deps[dep]]

                if is_deps_finalized and is_deps_active:
                    nodes[key].run(ctx, values)
                elif nodes[key].active:
                    nodes[key].run(ctx, values)
            cache[key] = nodes[key].output
    
    print_nodes_state(node_layers)

    print(bt.__dict__)
    print(cache)
    # for node_layer in node_layers:
    #     for key in node_layer:
    #         print(nodes[key].__dict__)
    

In [16]:
async def message():
    async with websockets.connect(cfg.config["eth_sepolia_ws_url"]) as ws:
        sub_newheads = { "id":1, "jsonrpc":"2.0", "method":"eth_subscribe", "params":["newHeads"] }
        await ws.send(json.dumps(sub_newheads))
        sub_res = await ws.recv()
        print(json.loads(sub_res))
        while True:
            message = await asyncio.wait_for(ws.recv(), timeout=60)
            block_info = json.loads(message)["params"]["result"]
            # start = time.time()
            await process_message(block_info)
            # end = time.time()
            # print(end-start)

In [17]:
import nest_asyncio
nest_asyncio.apply()

loop = asyncio.get_event_loop()
while True:
    loop.run_until_complete(message())

{'jsonrpc': '2.0', 'id': 1, 'result': '0x1e2503a7742e13680b2734069d17bfac'}
{'eth::sepolia': {'block_time': 5034196}}
######## node state ##########
[(False, True, None)]
[(False, True, 0.001), (False, True, 0.002)]
[(False, True, 0)]
[(False, True, None)]
[(False, True, 0.003)]
New transaction. tx1
Transaction successful! tx1
New transaction. tx2
Transaction successful! tx2
######## node state ##########
[(True, True, {})]
[(True, False, 0.001), (True, False, 0.002)]
[(False, True, 0)]
[(False, True, None)]
[(False, True, 0.003)]
{'start': 5034205, 'end': 5034245, 'frequency': 10, 'chain': 'eth::sepolia'}
{'data': {}, 'tx1': 0.001, 'tx2': 0.002, 'cum': 0, 'comp': None, 'tx3': 0.003}
{'eth::sepolia': {'block_time': 5034197}}
######## node state ##########
[(False, True, {})]
[(True, False, 0.001), (True, False, 0.002)]
[(False, True, 0)]
[(False, True, None)]
[(False, True, 0.003)]
######## node state ##########
[(False, True, {})]
[(True, False, 0.001), (True, False, 0.002)]
[(False, 

ConnectionClosedOK: received 1001 (going away) upstream went away; then sent 1001 (going away) upstream went away